🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Author: Laurent Sorba*

*Date: 04/07/2025*

**Description :**

This notebook filters and extracts public transportation data (GTFS format) according to a department number. In our case specifically for the Isère department (code 38) in France.
The notebook downloads GTFS (General Transit Feed Specification) data from various transportation providers and uses precise administrative boundaries to spatially filter bus stops and routes that operate within the department boundaries.

See https://github.com/data-for-good-grenoble/mobilite_durable/issues/13

## Définition des fonctions

In [ ]:
import json
from pathlib import Path

import geopandas as gpd
import gtfs_kit as gk
import pandas as pd
import requests


# 1. Télécharger et extraire les données GTFS
def download_gtfs_data(url):
    """Télécharge et extrait les données GTFS"""
    print(f"=== Télécharge les données GTFS depuis {url} ===")
    feed = gk.read_feed(url, dist_units="km")

    print(f"Chargé {len(feed.stops)} arrêts")
    print(f"Chargé {len(feed.routes)} lignes")
    print(f"Chargé {len(feed.stop_times)} stop_times")
    print(f"Chargé {len(feed.trips)} trips")

    return feed


# 2. Filtrer les arrêts par département de l'Isère par défaut
def filter_stops_by_department(feed, department_code="38"):
    """Filtre les arrêts par code département"""

    # 1. Charger les limites administratives du département
    department_boundary = download_department_boundary(department_code)

    if department_boundary is None:
        print(
            "Impossible de télécharger les limites administratives. Utilisation du filtrage par coordonnées."
        )
        exit()

    area_gdf = gpd.GeoDataFrame(
        {"department": [department_code]}, geometry=[department_boundary], crs="EPSG:4326"
    )

    # 2. Filtrer les arrêts qui se trouvent dans le département
    return gk.stops.get_stops_in_area(feed, area_gdf)


# 3. Filtrer les lignes de transport correspondant
def filter_routes_by_stops(routes_df, filtered_stops, trips_df=None, stop_times_df=None):
    """Filtre les lignes qui desservent les arrêts filtrés"""
    if trips_df is not None and stop_times_df is not None:
        # Trouver les trips qui passent par les arrêts filtrés
        department_stop_ids = filtered_stops["stop_id"].unique()
        trips_with_department_stops = stop_times_df[
            stop_times_df["stop_id"].isin(department_stop_ids)
        ]["trip_id"].unique()

        # Trouver les routes correspondantes
        routes_in_department = trips_df[trips_df["trip_id"].isin(trips_with_department_stops)][
            "route_id"
        ].unique()

        filtered_routes = routes_df[routes_df["route_id"].isin(routes_in_department)]
    else:
        # Si pas de données de correspondance, retourner toutes les routes
        filtered_routes = routes_df

    return filtered_routes


def download_department_boundary(department_code):
    """
    Charge ou télécharge les limites administratives du département de l'Isère
    """

    # Utiliser les données de data.gouv.fr
    try:
        boundary = load_department_geometry(department_code)
        if boundary is not None:
            return boundary
    except Exception as e:
        print(f"Erreur avec data.gouv.fr: {e}")

    # Fallback : Utiliser des coordonnées prédéfinies
    try:
        boundary = create_isere_boundary_from_coords()
        return boundary
    except Exception as e:
        print(f"Erreur avec coordonnées prédéfinies: {e}")

    return None


def download_from_data_gouv(file_path):
    """
    Télécharge depuis data.gouv.fr - Contours des départements français
    """
    # URL des contours des départements français
    url = "https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb"
    print(f"=== Télécharge les contours des départements français {url} ===")
    try:
        # Télécharger le GeoJSON
        response = requests.get(url, timeout=30)
        response.raise_for_status()

        # Récupérer le contenu JSON
        geojson_data = response.json()

        # Créer le répertoire s'il n'existe pas
        file_path = Path(file_path)
        file_path.parent.mkdir(parents=True, exist_ok=True)

        # Sauvegarder dans le fichier
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(geojson_data, f, ensure_ascii=False, indent=2)

        print(f"Fichier sauvegardé avec succès : {file_path}")
        print(f"Nombre de départements : {len(geojson_data['features'])}")

    except Exception as e:
        print(f"Erreur lors du téléchargement depuis data.gouv.fr: {e}")
        return None


def load_department_geometry(department_code="38"):
    """
    Télécharge depuis data.gouv.fr - Contours des départements français
    """
    # URL des contours des départements français
    file = "../data/transportdatagouv/contour-des-departements.geojson"
    print(f"=== Charge les contours des départements français depuis le fichier {file} ===")

    try:
        # Test if file exists
        if not Path(file).is_file():
            print(f"-> Le fichier n'existe pas, téléchargement nécessaire.")
            download_from_data_gouv(file)

        # Charger le GeoJSON
        france_departments = gpd.read_file(file)

        # Filtrer sur un département
        department_boundary = france_departments[france_departments["code"] == department_code]

        if not department_boundary.empty:
            return department_boundary.iloc[0].geometry
        else:
            print(f"Département {department_code} non trouvé dans les données")
            return None

    except Exception as e:
        print(f"Erreur lors du chargement des contours des départements: {e}")
        return None


def create_isere_boundary_from_coords():
    """
    Crée une approximation des limites de l'Isère à partir de coordonnées connues
    """
    from shapely.geometry import Polygon

    print(f"=== Fallback: Utiliser des coordonnées prédéfinies de l'Isère ===")

    # Coordonnées approximatives des limites de l'Isère
    isere_coords = [
        (5.2, 44.8),  # Sud-Ouest
        (6.3, 44.8),  # Sud-Est
        (6.3, 45.9),  # Nord-Est
        (5.2, 45.9),  # Nord-Ouest
        (5.2, 44.8),  # Fermeture du polygone
    ]

    boundary = Polygon(isere_coords)
    return boundary

## Définition des variables

In [ ]:
pd.set_option("display.max_columns", None)

# URLs des données GTFS contenant l'Isère
gtfs_sources = {
    # Definition https://transport.data.gouv.fr/datasets/agregat-oura
    "agregat-oura": "https://api.oura3.cityway.fr/dataflow/offre-tc/download?provider=OURA&dataFormat=GTFS&dataProfil=OPENDATA",
    # Definition https://transport.data.gouv.fr/datasets/reseau-cars-region-isere-38
    "reseau-cars-region-isere-38": "https://www.itinisere.fr/fr/donnees-open-data/169/OpenData/Download?fileName=CG38.GTFS.zip",
    # Definition https://transport.data.gouv.fr/datasets/horaires-theoriques-du-reseau-tag
    "TAG_Grenoble": "https://data.mobilites-m.fr/api/gtfs/SEM",
}

choose_department_code = "38"
data_set = "agregat-oura"
gtfs_url = gtfs_sources[data_set]

In [ ]:
print(
    f"=== Filtrage avec limites administratives du '{choose_department_code}' sur '{data_set}' ==="
)

### Télécharger les données GTFS

In [ ]:
result_feed = download_gtfs_data(gtfs_url)

### Filtrer les arrêts

In [ ]:
result_filtered_stops = filter_stops_by_department(result_feed, choose_department_code)
print(
    f"Nombre d'arrêts dans le {choose_department_code}: {len(result_filtered_stops)} sur un total de {len(result_stops_df)}"
)

### Filtrer les lignes

In [ ]:
result_filtered_routes = filter_routes_by_stops(
    result_feed.routes, result_filtered_stops, result_feed.trips, result_feed.stop_times
)
print(
    f"Nombre de lignes dans le {choose_department_code}: {len(result_filtered_routes)} sur un total de {len(result_feed.routes)}"
)

## Résultats

In [ ]:
# Exemples d'arrêts
print("\nPremiers arrêts filtrés:")
print(
    result_filtered_stops[["stop_id", "stop_code", "stop_name", "stop_lat", "stop_lon"]].head(
        10
    )
)

In [ ]:
# Exemples de lignes
print("\nPremières lignes filtrées:")
print(result_filtered_routes[["route_id", "route_short_name", "route_long_name"]].head(10))